In [5]:
using Plots, ForwardDiff,LinearAlgebra,Roots, BenchmarkTools

## Funktionendefinitionen 
Definition der Funktionen $M_{an}$, $m$ und $\nabla (S - \langle m,m_p \rangle)= m - m_p$ 

Außerdem wird das Problem auf dem Kreis parametrisiert. Wir haben $$\nabla Prob(r \sin \phi \cos \psi,r \sin \phi \sin \psi,r \cos \phi) = (m(r \sin \phi \cos \psi,r \sin \phi \sin \psi,r \cos \phi)-m_p) J_{r,\phi,\psi}(r \sin \phi \cos \psi,r \sin \phi \sin \psi,r \cos \phi)$$

In [15]:

Man(hr,ms,A) = 2ms/π * atan(hr/A)
m(hr,ms,A) = Man(norm(hr),ms,A) * hr ./ norm(hr)
m(hr) = m(hr,1.23*10^3,38)
∇Prob(u,mp) = m(u) .- mp
transformToEuklidean(ϕ,θ,r,h) =  h + [r * sin(θ) * cos(ϕ),r * sin(θ) * sin(ϕ),r * cos(θ)]

∇ProbOnBall(ϕ,θ,r,h,mp) = (m(transformToEuklidean(ϕ,θ,r,h)) .- mp)' * J(r,ϕ,θ)
∇ProbOnBall(u,r,h,mp) = ∇ProbOnBall(u...,r,h,mp)

∇ProbOnBall (generic function with 2 methods)

## Newton Verfahren

In [16]:
function newton(f,x,eps = 10^-15,maxiter = 200)
   
    for i in 1:maxiter
        
        J = ForwardDiff.jacobian(f,x)
        xn = (J \ -f(x)) + x
        if(norm(xn -x) < eps)
            return xn
        end
        x = xn
    end
    print("Reached MaxIter, solution is maybe not convergent")
    return x
end

newton (generic function with 3 methods)

## Jacobi Matrix der Transformation

In [17]:
J(r,ϕ,θ) = 
    [[sin(θ)*cos(ϕ), r * cos(θ) * cos(ϕ), -r * sin(θ) * sin(ϕ)],
    [sin(θ)* sin(ϕ),r * cos(θ) * sin(ϕ),r * sin(θ) * cos(ϕ)],
    [cos(θ), -r * sin(θ),0]]

J (generic function with 1 method)

We now solve $$\arg\min_{u \in \mathring K(h)} S(u) - \langle u, m_p \rangle$$ with the newton method. If the solution is not in $\mathring K(h)$, then  we solve on the ball $\partial K(h)$. For this we set $g(\theta,\phi) = r \sin(θ)  \cos(ϕ),r  \sin(θ) \sin(ϕ),r \cos(θ)$ and solve $$\nabla(S(g(\theta,\phi)) - <m,g(\theta,\phi)> = 0$$

In [1]:
struct ProblemStructure
    χ ::Real
    h ::Vector
    mp ::Real
    Prob :: Function
    ∇Prob :: Function
    ∇ProbOnBall :: Function

    function ProblemStructure(χ,h,mp,Prob,∇Prob)
        ∇ProbOnBall(ϕ,θ,r,h,mp) = ∇Prob(transformToEuklidean(ϕ,θ,r,h),h,mp)' * J(r,ϕ,θ)
        ProblemStructure(
            χ,
            h,
            mp,
            Prob,
            ∇Prob,
            (u,r,h,mp) -> ∇ProbOnBall(u...,r,h,mp)
        )
    end
end

function solver(s ::ProblemStructure)
    
    hr = newton((u) -> ∇Prob(u,s.h,s.mp),[0.1,0.1,0.1])

    if(norm(hr .- s.h) >= s.χ)
        #println("Solution is not in K(h), searching on the edge instead")

        ans = newton((u) -> ∇ProbOnBall(u...,χ,s.h,s.mp),[0.4,π-0.5])
        hr = transformToEuklidean(ans...,s.χ,s.h)
    end
    return hr
end

solver(χ,h,mp) = solve(ProblemStructure(χ,h,mp))

solver (generic function with 2 methods)

In [38]:
prob

BenchmarkTools.Trial: 9008 samples with 5 evaluations.
 Range (min … max):    6.006 μs … 863.051 μs  ┊ GC (min … max):  0.00% … 66.63%
 Time  (median):     104.188 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   110.657 μs ±  84.032 μs  ┊ GC (mean ± σ):  12.43% ± 14.75%

  █           █▇▅▄▃▄▇▅▄▂          ▁▁▁▁▁                         ▂
  █▅▁▁▁▁▁▁▁▁▁▁███████████▆▇▆▆▆▆▆▇███████▇▆▅▆▆▆▆▅▇▆▆▇▇▇▇▇▆▆▆▇▇▆▆ █
  6.01 μs       Histogram: log(frequency) by time        458 μs <

 Memory estimate: 6.08 KiB, allocs estimate: 76.

3.6736736736736737